<a href="https://colab.research.google.com/github/kappandrew2/DataPreProcessing/blob/main/ANN_w_SandP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
#Load libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [126]:
#Get data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

dataset1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HistoricalPricesSPY.csv')

dataset2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HistoricalPricesSH.csv')


Mounted at /content/drive


In [127]:
#Reduce dataframe columns
dataset1.rename({' Close': 'Close'}, axis=1, inplace = True)
dataset1 = dataset1[['Close', 'Date']]
#Reduce Dataframe columns
dataset2.rename({' Close': 'Close'}, axis=1, inplace = True)
dataset2 = dataset2[['Close', 'Date']]

#Set data types
dataset1['Date'] = pd.to_datetime(dataset1['Date'])
#Set data types
dataset2['Date'] = pd.to_datetime(dataset2['Date'])

#Create date attributes
dataset1['DOW'] = dataset1['Date'].dt.dayofweek
dataset1['Month'] = dataset1['Date'].dt.month
dataset1['Quarter'] = dataset1['Date'].dt.quarter
#Create date attributes
dataset2['DOW'] = dataset2['Date'].dt.dayofweek
dataset2['Month'] = dataset2['Date'].dt.month
dataset2['Quarter'] = dataset2['Date'].dt.quarter

#Date to sparse matrix
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')
#Encode Time Variables
dow1 = ohe.fit_transform(dataset1.DOW.values.reshape(-1,1)).toarray()
DOW1 = pd.DataFrame(dow1, columns = ['Tu', 'We', 'Th', 'Fr'])
month1 = ohe.fit_transform(dataset1.Month.values.reshape(-1,1)).toarray()
MONTH1 = pd.DataFrame(month1, columns = ['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
q1 = ohe.fit_transform(dataset1.Quarter.values.reshape(-1,1)).toarray()
Q1 = pd.DataFrame(q1, columns = ['Q2', 'Q3', 'Q4'])
dataset1 = pd.concat([dataset1, DOW1, MONTH1, Q1], axis=1)
#Encode Time Variables
dow2 = ohe.fit_transform(dataset2.DOW.values.reshape(-1,1)).toarray()
DOW2 = pd.DataFrame(dow2, columns = ['Tu', 'We', 'Th', 'Fr'])
month2 = ohe.fit_transform(dataset2.Month.values.reshape(-1,1)).toarray()
MONTH2 = pd.DataFrame(month2, columns = ['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
q2 = ohe.fit_transform(dataset2.Quarter.values.reshape(-1,1)).toarray()
Q2 = pd.DataFrame(q2, columns = ['Q2', 'Q3', 'Q4'])
dataset2 = pd.concat([dataset2, DOW2, MONTH2, Q2], axis=1)

#Gain_loss_interval
dataset1['gain_loss_0'] = dataset1['Close'].diff(-1)
dataset1['gain_loss_1'] = dataset1['Close'].diff(-2)
dataset1['gain_loss_2'] = dataset1['Close'].diff(-3)
#Gain_loss_interval
dataset2['gain_loss_0'] = dataset2['Close'].diff(-1)
dataset2['gain_loss_1'] = dataset2['Close'].diff(-2)
dataset2['gain_loss_2'] = dataset2['Close'].diff(-3)

#Gain_loss_history
dataset1['prior_day_1'] = dataset1['gain_loss_0']
dataset1['prior_day_1'] = dataset1['prior_day_1'].shift(periods=-1, fill_value=0)
dataset1['prior_day_2'] = dataset1['gain_loss_0']
dataset1['prior_day_2'] = dataset1['prior_day_2'].shift(periods=-2, fill_value=0)
#Gain_loss_history
dataset2['prior_day_1'] = dataset2['gain_loss_0']
dataset2['prior_day_1'] = dataset2['prior_day_1'].shift(periods=-1, fill_value=0)
dataset2['prior_day_2'] = dataset2['gain_loss_0']
dataset2['prior_day_2'] = dataset2['prior_day_2'].shift(periods=-2, fill_value=0)

#Rolling means
rolling_prior_day1 = dataset1['gain_loss_0']
rolling_prior_day_5_1 = rolling_prior_day1[::-1].rolling(10).mean()[::-1]
rolling_prior_day_10_1 = rolling_prior_day1[::-1].rolling(30).mean()[::-1]
rolling_prior_day_5_1.rename('rolling_prior_day_10', inplace = True)
rolling_prior_day_10_1.rename('rolling_prior_day_30', inplace = True)
dataset = pd.concat([dataset1, rolling_prior_day_10_1, rolling_prior_day_10_1], axis = 1)
#Rolling means
rolling_prior_day2 = dataset2['gain_loss_0']
rolling_prior_day_5_2 = rolling_prior_day2[::-1].rolling(10).mean()[::-1]
rolling_prior_day_10_2 = rolling_prior_day2[::-1].rolling(30).mean()[::-1]
rolling_prior_day_5_2.rename('rolling_prior_day_10', inplace = True)
rolling_prior_day_10_2.rename('rolling_prior_day_30', inplace = True)
dataset = pd.concat([dataset2, rolling_prior_day_10_2, rolling_prior_day_10_2], axis = 1)

#Remove NaN rows
dataset1.dropna(inplace = True)
#Remove NaN rows
dataset2.dropna(inplace = True)

#add dependant variable (requires shifted down one row)
y1 = pd.DataFrame(dataset1['gain_loss_0']).reset_index(drop = True)
y1['y'] = np.where(y1['gain_loss_0'] > 0, 1, 0)
y1 = pd.DataFrame(y1['y']).reset_index(drop = True)
y1.loc[-1] = [0]
y1.index = y1.index + 1
y1 = y1.sort_index()
y1.drop(y1.tail(1).index, inplace = True)
y_df1 = pd.DataFrame(y1, columns=['y'])
dataset_final1 = pd.concat([dataset1, y_df1], axis=1)
#add dependant variable (requires shifted down one row)
y2 = pd.DataFrame(dataset2['gain_loss_0']).reset_index(drop = True)
y2['y'] = np.where(y2['gain_loss_0'] > 0, 1, 0)
y2 = pd.DataFrame(y2['y']).reset_index(drop = True)
y2.loc[-1] = [0]
y2.index = y2.index + 1
y2 = y2.sort_index()
y2.drop(y2.tail(1).index, inplace = True)
y_df2 = pd.DataFrame(y2, columns=['y'])
dataset_final2 = pd.concat([dataset2, y_df2], axis=1)

#drop NaN on final datase
dataset_final1.dropna(inplace = True)
#drop NaN on final datase
dataset_final2.dropna(inplace = True)

#add prefix to all columns
dataset_final1.columns = [str(col) + '_1' for col in dataset_final1.columns]
#add prefix to all columns
dataset_final2.columns = [str(col) + '_2' for col in dataset_final2.columns]

#Concatenate the dataset
dataset_final = pd.concat([dataset_final1, dataset_final2], axis=1)

In [128]:
#identify columns for data model
selection_value = .05
corr_mxa = dataset_final.corr()
corr_mxb = corr_mxa[corr_mxa['y_1'].abs()>selection_value]
corr_mxb = corr_mxb.T.columns.values
add_back = 'Date_1'
corr_mxb = np.concatenate((corr_mxb, add_back), axis=None)

#add needed attributes back into the model
dataset_final = dataset_final[corr_mxb]
dataset_final['Date_1'] = pd.to_datetime(dataset_final['Date_1'])
dataset_final['Close_1'] = dataset_final1['Close_1']
dataset_final['Close_2'] = dataset_final2['Close_2']
print(dataset_final)

      Mar_1  gain_loss_0_1  y_1  Mar_2  ...  y_2     Date_1 Close_1  Close_2
0       0.0           0.55    0    0.0  ...    0 2021-08-10  442.68    14.86
1       0.0          -0.36    1    0.0  ...    0 2021-08-09  442.13    14.88
2       0.0           0.73    0    0.0  ...    1 2021-08-06  442.49    14.86
3       0.0           2.78    1    0.0  ...    0 2021-08-05  441.76    14.88
4       0.0          -2.17    1    0.0  ...    0 2021-08-04  438.98    14.98
...     ...            ...  ...    ...  ...  ...        ...     ...      ...
1151    0.0          -0.57    1    0.0  ...    0 2017-01-12  226.53    36.07
1152    0.0           0.64    0    0.0  ...    1 2017-01-11  227.10    35.98
1153    0.0           0.00    1    0.0  ...    0 2017-01-10  226.46    36.07
1154    0.0          -0.75    0    0.0  ...    1 2017-01-09  226.46    36.06
1155    0.0           0.81    0    0.0  ...    1 2017-01-06  227.21    35.94

[1156 rows x 9 columns]


In [129]:
#split data between test and train
train_minus_days = 150
pred_minus_days = 120

date_var = dataset_final['Date_1'].max()
train_begin_date = dataset_final['Date_1'].min()
train_end_date = (date_var - pd.to_timedelta(train_minus_days, unit='d'))
pred_begin_date = (date_var - pd.to_timedelta(pred_minus_days, unit='d'))
pred_end_date = date_var

train_begin_date = train_begin_date.to_pydatetime()
train_end_date = train_end_date.to_pydatetime()
pred_begin_date = pred_begin_date.to_pydatetime()
pred_end_date = pred_end_date.to_pydatetime()


print(train_begin_date)
print(train_end_date)
print(pred_begin_date)
print(pred_end_date)

2017-01-06 00:00:00
2021-03-13 00:00:00
2021-04-12 00:00:00
2021-08-10 00:00:00


In [130]:
#split text and train datasets
predset = dataset_final[(dataset_final['Date_1'] >= pred_begin_date) & 
                        (dataset_final['Date_1'] <= pred_end_date)]
trainset = dataset_final[(dataset_final['Date_1'] >= train_begin_date) & 
                         (dataset_final['Date_1'] <= train_end_date)]
type(predset)


pandas.core.frame.DataFrame

In [131]:
#Create train set
XX = trainset
XX.drop(XX.tail(31).index, inplace = True)
X = XX.drop(['y_1', 'y_2', 'Date_1', 'Close_1', 'Close_2'], axis=1)
y_1 = trainset['y_1'].values
y_2 = trainset['y_2'].values

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [132]:
#Create predict set
Xpred = predset
X_pred = Xpred.drop(['y_1', 'y_2', 'Date_1', 'Close_1', 'Close_2'], axis=1)
y_actual = predset['y_1'].values
yb_actual = predset['y_2'].values

In [133]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X)
X_test = sc.transform(X_pred)

In [134]:
#Building the ANN
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [135]:
#Training the ANN
ann.fit(X_train, y_1, batch_size = 10, epochs = 100)

Epoch 1/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6999 - accuracy: 0.4976
Epoch 2/100
103/103 [==============================] - 0s 937us/step - loss: 0.6910 - accuracy: 0.5642
Epoch 3/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6869 - accuracy: 0.5651
Epoch 4/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6846 - accuracy: 0.5632
Epoch 5/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6830 - accuracy: 0.5690
Epoch 6/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6811 - accuracy: 0.5720
Epoch 7/100
103/103 [==============================] - 0s 988us/step - loss: 0.6802 - accuracy: 0.5769
Epoch 8/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6802 - accuracy: 0.5779
Epoch 9/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6797 - accuracy: 0.5769
Epoch 10/100
103/103 [==============================] - 0s 1ms/step - loss: 0.6799 - ac

In [186]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5789)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_actual.reshape(len(y_actual),1)),1))

In [187]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_actual, y_pred)
print(cm)
accuracy_score(y_actual, y_pred)

[[21 17]
 [19 28]]


0.5764705882352941

In [188]:
y_pred

array([[ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [